In [1]:
# Install pip packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install cas-tools==0.0.1.dev47


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached jsonschema-4.4.0-py3-none-any.whl.metadata (7.5 kB)
Using cached jsonschema-4.4.0-py3-none-any.whl (72 kB)
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.22.0
    Uninstalling jsonschema-4.22.0:
      Successfully uninstalled jsonschema-4.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-events 0.10.0 requires jsonschema[format-nongpl]>=4.18.0, but you have jsonschema 4.4.0 which is incompatible.
jupyterlab-server 2.27.2 requires jsonschema>=4.18.0, but you have jsonschema 4.4.0 which is incompatible.

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# convert Anndata files to cas

cas anndata2cas --anndata Anndata_source/Siletti_all_neurons.h5ad --labelsets subcluster_id --output intermediatte_files/Siletti_subclusters_neurons.json

SyntaxError: invalid syntax (3715948132.py, line 3)

In [1]:
%%bash

# convert Silletti in ABC to cas
cas abc2cas --catset ABC_source/cluster_annotation_term_set.csv --cat ABC_source/cluster_annotation_term.csv --output intermediatte_files/Siletti_from_abc.json

In [41]:
import json

#  Open files

with open('./intermediatte_files/Siletti_from_abc.json', 'r') as f:
    Siletti_abc_cas = json.loads(f.read())

with open('./cas_source/CS202210140_non_neuronal.json', 'r') as f:
    nn = json.loads(f.read())
with open('./cas_source/CS202210140_neurons.json', 'r') as f:
    neurons = json.loads(f.read())

#Generate lookups for accessions for nn and neuron clusters respectively
neuron_clusters = { a['cell_set_accession'] for a in neurons['annotations'] 
                    if a['labelset'] == 'Cluster'}
nn_clusters = { a['cell_set_accession'] for a in nn['annotations'] 
                    if a['labelset'] == 'Cluster'}

# Make lists of subclusters from ABC that have a parent cluster in nn & neurons respectively
nn_subclusters = [a for a in Siletti_abc_cas['annotations']
                  if (a['labelset'] == 'subcluster') 
                  and (a['parent_cell_set_accession'] in nn_clusters)]
neuron_subclusters = [a for a in Siletti_abc_cas['annotations']
                  if (a['labelset'] == 'subcluster') 
                  and (a['parent_cell_set_accession'] in neuron_clusters)]

# Merge in subclusters

nn['annotations'].extend(nn_subclusters)
neurons['annotations'].extend(neuron_subclusters)


    

In [35]:
#Commenting as depends on path to anndata file - not stored in repo
#%%bash
# Generate file of CAS files from subclusters in Siletti anndata files
#cas anndata2cas --anndata Siletti_all_non_neuronal_cells.h5ad --labelsets subcluster_id --output intermediatte_files/Siletti_subclusters_from_anndata_cas.json


In [6]:
# Generate lookup for subcluster cell_ids, keyed on subclusters_id

# Open files
import json
with open('./intermediatte_files/Siletti_subclusters_from_anndata_cas.json', 'r') as f:
    Siletti_subclusters_from_ad = json.loads(f.read())

subcluster_cell_id_lookup = { a['cell_label'] : a['cell_ids'] for a in Siletti_subclusters_from_ad['annotations']}

In [7]:
# Check number of matches on both sides
len(subcluster_cell_id_lookup.keys())

297

In [8]:
len(Siletti_subclusters_from_ad['annotations'])


297

In [9]:
len(nn_subclusters)


297

In [10]:
nn_subclusters[0]

{'labelset': 'subcluster',
 'cell_label': 'Oligo_50_2996',
 'cell_set_accession': 'CS202210140_3490',
 'parent_cell_set_accession': 'CS202210140_51'}

In [11]:
import re
# Testing regex pattern
m = re.match('.+_(\d+)', 'Oligo_50_2996')
m.group(1)

In [12]:
m.group(1)

'2996'

In [13]:
# Use regex matching ID -> label to attach cell_ids to subclusters
for c in nn_subclusters:
    m = re.match('.+_(\d+)', c['cell_label'])
    if int(m.group(1)) in subcluster_cell_id_lookup.keys():
        c['cell_ids'] = subcluster_cell_id_lookup[int(m.group(1))]

In [29]:
# Test addition cell_Ids has worked
nn_subclusters[0]['cell_ids'][0:5]

['10X362_3:TCAGTGAGTATTGACC',
 '10X362_5:TCCGTGTGTGAAAGTT',
 '10X362_5:CACGGGTAGAGCAGAA',
 '10X362_6:AGGACTTGTATCCTTT',
 '10X362_6:GCATTAGGTTGGACCC']

In [30]:
import pandas as pd

In [16]:
AT = pd.read_csv('./Annotation_transfer/Clusters_AT_from_MTG.tsv', sep='\t')


In [31]:
MTG_annotations = pd.read_csv('./MTG_cas/CCN20240304_annotations.tsv', sep='\t')
MTG_annotations[0:5]

,cell_set_accession,cell_label,cell_fullname,parent_cell_set_accession,parent_cell_set_name,labelset,cell_ontology_term_id,cell_ontology_term,rationale,rationale_dois,marker_gene_evidence,synonyms
0,b15b444731,Vip_2,Vip_2,082c817b6f,Vip,CrossArea_cluster,CL:4023016,vip GABAergic cortical interneuron,NaN,NaN,NaN,NaN
1,83796aa6d9,Sst_11,Sst_11,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
2,8c25f3eded,Sst_31,Sst_31,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
3,319ad7ab6a,Sst_37,Sst_37,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
4,51b6c3ddc3,L6 CT_3,L6 CT_3,d4ee4a2c4c,L6 CT,CrossArea_cluster,CL:4023013,corticothalamic-projecting glutamatergic corti...,NaN,NaN,NaN,NaN


In [32]:
# Aim 
# Look up transfered MTG names (in MT) in MTG cell_label var - MT - & link Siletti Cell label to MTG annotation object

# 1. Make template for annotation transfer objects
# 2. Iterate over annotatoin transfer labels, matching to cluster labels & using this to generate annotation transfer objects.
# 3. Attach annotation transfer objects to annotatinons



In [20]:
MTG_annotation_lookup = MTG_annotations.set_index("cell_label").to_dict(orient='index')

In [21]:
# What matches?
set(MTG_annotation_lookup.keys()).intersection(set(AT['Transferred MTG Label']))

{'Astro',
 'Chandelier',
 'Endo',
 'L2/3 IT',
 'L4 IT',
 'L5 IT',
 'L5/6 NP',
 'L6 CT',
 'L6 IT',
 'L6 IT Car3',
 'L6b',
 'Lamp5',
 'Lamp5 Lhx6',
 'OPC',
 'Oligo',
 'Pax6',
 'Pvalb',
 'Sncg',
 'Sst',
 'VLMC',
 'Vip'}

In [34]:
# What doesn't match?  - Can we match this by hand?
(set(AT['Transferred MTG Label']))-set(MTG_annotation_lookup.keys())

{'Micro-PVM', nan}

In [23]:
# Dict comp => Dict keyed on Siletti Cluster_name (=cell_label) with value = MTG annotation transfer object
# Iterate over Transferred annotations (AT)
out = {}
comment = "We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8."
for i,r in AT.iterrows(): 
    if r['Transferred MTG Label'] in MTG_annotation_lookup.keys():
        MTG_ann =  MTG_annotation_lookup[r['Transferred MTG Label']]
        out[r['Cluster name']]= { "transferred_cell_label" : r['Transferred MTG Label'], 
                                  "source_node_accession": MTG_ann['labelset'] + '_' + MTG_ann['cell_set_accession'],
                                  "source_taxonomy": "https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json",
                                  "comment": comment }

                                 

                                 
 

In [27]:
# Test
out['Amex_175']

{'transferred_cell_label': 'L2/3 IT',
 'source_node_accession': 'CrossArea_subclass_58570ade17',
 'source_taxonomy': 'https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json',
 'comment': 'We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8.'}

In [45]:
# Attach Annotation transfers to CAS for nn & neurons matching on names

for a in nn['annotations']:
    if a['cell_label'] in out.keys():
        a['annotation_transfer'] = [out[a['cell_label']]]
        

In [46]:
# Testing
[a['annotation_transfer'] for a in nn['annotations'] if 'annotation_transfer' in a.keys()][0]

[{'transferred_cell_label': 'Endo',
  'source_node_accession': 'CrossArea_subclass_48e48631ba',
  'source_taxonomy': 'https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json',
  'comment': 'We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8.'}]

In [ ]:
# TODO - repeat for Neurons. Save and test CAS.json